# Driver and engine performance impact on F1 race results

Łukasz Filo, Klaudiusz Grobelski

## Problem formulation [0-5 pts]:

- is the problem clearly stated [1 pt]
- what is the point of creating model, are potential use cases defined [1 pt]
- where do data comes from, what does it containt [1 pt]
- DAG has been drawn [1 pt]
- confoundings (pipe, fork, collider) were described [1 pt]

In this notebook, we will develop a Bayesian multilevel binomial regression model to predict driver performance across recent Formula 1 seasons. Specifically, we will use data from the 2020–2024 seasons. The input data includes information about drivers, constructors, and engine suppliers used by each team. The primary objective of this analysis is to understand and predict how various factors—such as driver skill, team changes, and engine suppliers—affect driver performance over time. This model could assist teams in making strategic decisions, such as evaluating whether changes in drivers or engine suppliers might enhance performance. It may also be valuable to fans and analysts seeking to assess the relative impact of technical and human factors on race results.

We use historical race data sourced from FastF1, which includes finishing positions, lap times, and team-driver pairings for each race. Driver skill ratings are obtained from the EA Sports F1 game, while engine usage data (i.e., which power unit each constructor used in a given season) is collected from Wikipedia, covering the 2020–2024 period.

To gain a clearer understanding of the relationships between variables and to identify potential sources of bias, we will construct a Directed Acyclic Graph (DAG).

In [ ]:
from IPython.display import Image
Image("images/DAG.png")

Why these data were used:

**EA Driver Rating** – an independent, available measure of skill that helps avoid subjective assessments.

**Engine, Constructor** – technical foundations influencing the result, with a clear causal meaning.

**Year + Constructor** – capturing variability between seasons.

We omitted data such as weather, which could cause irregular effects on the model, since races in difficult conditions are hard to predict. The number of pit stops was also excluded because, in a typical race, most teams perform the same number of pit stops, or there are cases where a driver can earn an extra point for the fastest lap. If the team knows they won’t catch the driver ahead or lose position, they may perform this tactical maneuver. However, an increased number of pit stops often occurs due to collisions during the race, meaning the driver might have more pit stops but a significantly lower position. This could confuse the model’s behavior.

DAGs illustrate the main types of confounding structures:

- **Pipe:** e.g., `Engine → Constructor_Performance → Race_Position` – the engine affects the constructor’s performance, which in turn influences the race position.

- **Fork:** e.g., `Year → EA_Driver_Rating` and `Year → Constructor` – the year influences the driver rating (if drivers performed well that year, they have a higher rating) and also affects constructors due to changing vehicle regulations each season.

- **Collider:** `Alpha_driver → Race_Position ← Constructor_Performance` – race outcome depends on both the team’s quality building the car and the skill of the driver racing it.


In [ ]:
import numpy as np
from cmdstanpy import CmdStanModel
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import arviz as az
import seaborn as sns
from scipy.special import expit
from numpy.random import binomial

# import logging
# logging.getLogger("cmdstanpy").setLevel(logging.WARNING)

## Data preprocessing [0-2 pts]:
- is preprocessing step clearly described [1 pt]
- reasoning and types of actions taken on the dataset have been described [1 pt]

In [ ]:
df = pd.read_csv('data/processed_data/data.csv')
unique_drivers = df['DriverId'].unique()
driver_id_map = {driver: idx + 1 for idx, driver in enumerate(unique_drivers)}
df['DriverId'] = df['DriverId'].map(driver_id_map)
drivers = df['DriverId'].values

unique_team = df['TeamId'].unique()
team_id_map = {team: idx + 1 for idx, team in enumerate(unique_team)}
df['TeamId'] = df['TeamId'].map(team_id_map)
teams = df['TeamId'].values

unique_engine = df['Engine'].unique()
engine_id_map = {engine: idx + 1 for idx, engine in enumerate(unique_engine)}
df['Engine'] = df['Engine'].map(engine_id_map)
engines = df['Engine'].values

unique_season = df['Season'].unique()
season_id_map = {season: idx + 1 for idx, season in enumerate(unique_season)}
df['Season'] = df['Season'].map(season_id_map)
seasons = df['Season'].values


df['Rating_all'] = (df['Rating'] - df['Rating'].mean()) / df['Rating'].std()

In [ ]:
def standardize_group(group):
    mean = group['Rating'].mean()
    std = group['Rating'].std()
    group['Rating_by_year'] = (group['Rating'] - mean) / std
    return group

df = df.groupby('Season', group_keys=False, observed=True).apply(standardize_group)
ratings = df['Rating_all'].values
ratings_by_year = df["Rating_by_year"].values
df['Position'] = df['Position'].astype(int)

In our model, we applied two approaches to standardizing driver rating data. In the first case, standardization was performed on the entire dataset, while in the second, it was done separately for each season.

In [ ]:
order_col = ['DriverId', 'Rating_by_year', 'Rating_all', 'TeamId', 'Engine', 'Season','Position']
df = df[order_col]
df.head()

## Model [0-4 pts]
- are two different models specified [1 pt]
- are difference between two models explained [1 pt]
- is the difference in the models justified (e.g. does adding aditional parameter makes sense? ) [1 pt]
- are models sufficiently described (what are formulas, what are parameters, what data are required ) [1 pt]

### Model 1  
  $$
  \text{model} = \alpha_{\text{constructor}} - \alpha_{\text{driver}} \cdot \text{Driver Rating}
  $$

  $$
  \theta = \mathrm{inv\_logit}(\text{model}) = \frac{1}{1 + e^{-\text{model}}}
  $$

  $$
  \text{position} \sim \mathrm{Binomial}(n=19, p=\theta)
  $$


In [ ]:
Image("images/model1.png")

### Parameters

- **Constructor** – represents the strength of the team. It can be interpreted as a deviation from the average constructor:

    - **Negative values** → better than average car  
    - **Positive values** → worse than average car  

- **Alpha_Driver** - Coefficient determining how much impact the driver has on the performance.

- **Driver_Rating** - Driver's rating, taken from the EA Sports F1 video game.


### Model 2
  $$
  \text{model} = \alpha_{\text{engine}} + \alpha_{\text{year\_constructor}} - \alpha_{\text{driver}} \cdot \text{Driver Rating}
  $$

  $$
  \theta = \mathrm{inv\_logit}(\text{model}) = \frac{1}{1 + e^{-\text{model}}}
  $$

  $$
  \text{position} \sim \mathrm{Binomial}(n=19, p=\theta)
  $$


In [ ]:
Image("images/model2.png")

### Parameters

- **Constructor** - Represents the team's strength for a given season. It can be interpreted as a deviation from the average constructor:

    - **Negative values** → better than average car  
    - **Positive values** → worse than average car  

- **Engine** - Represents the engine used:
    - **Negative values** → better than average engine  
    - **Positive values** → worse than average engine  


- **Alpha_Driver** - Coefficient determining how much impact the driver has on the performance.

- **Driver_Rating** - Driver's rating, taken from the EA Sports F1 video game.


### Difference between the models

The difference between the first and the second model is based on two main factors. The first is the inclusion of information about the engine manufacturer used by each team. In Formula 1, there are several engine suppliers, and incorporating this parameter allows the analysis to assess how a change in engine supplier affected a team's performance. The second distinguishing factor is the inclusion of temporal parameters. In the first model, we assume that a team is generally strong or weak, whereas in the second model, we account for variations in team performance across different seasons. A well-known example for any fan is the Mercedes team, which dominated from 2014 to 2021 by winning the Constructors' Championships, but has seen a decline in performance since the 2022 season. Differences in how data is fed into the models also result from the different standardization methods used.

### Required Data

- Team performance data across multiple seasons (e.g., race results).  
- Identification of constructor for each team and season.  
- Engine manufacturer information for each team and season.
- Driver rating data taken from the F1 game developed by EA Sports.

## Priors [0-4 pts]
- Is it explained why particular priors for parameters were selected [1 pt]
- Have prior predictive checks been done for parameters (are parameters simulated from priors make sense) [1 pt]
- Have prior predictive checks been done for measurements (are measurements simulated from priors make sense) [1 pt]
- How prior parameters were selected [1 pt]

## The Prior tests were prepared for the best, average, and weakest driver.

### Model 1 PPC

In [ ]:
model_1_ppc = CmdStanModel(stan_file='stan/model_1_ppc.stan')

In [ ]:
def draw_plots_ppc_model_1(sigmas, driver_rating):
    fig, axes = plt.subplots(3, 4, figsize=(16, 10))
    colors = ["#130582", "#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"]

    for s_i in range(3):
        sigma = {'sigma': sigmas[s_i], 'driver_rating': driver_rating}
        model_1_ppc_sim = model_1_ppc.sample(data=sigma, iter_warmup=1, fixed_param=True, seed=25062025)
        
        axes[s_i, 0].hist(model_1_ppc_sim.stan_variable('alpha_driver').flatten(), bins=100, density=True, color=colors[1], alpha=0.8)
        axes[s_i, 0].set_yticks([])
        axes[s_i, 0].set_title(f'alpha_driver ~ Normal(1, {sigmas[s_i]})')

        axes[s_i, 1].hist(model_1_ppc_sim.stan_variable('constructor').flatten(), bins=100, density=True, color=colors[2], alpha=0.8)
        axes[s_i, 1].set_yticks([])
        axes[s_i, 1].set_title(f'constructor ~ Normal(0, {sigmas[s_i]})')

        axes[s_i, 2].hist(model_1_ppc_sim.stan_variable('theta').flatten(), bins=100, density=True, color=colors[4], alpha=0.8)
        axes[s_i, 2].set_yticks([])
        axes[s_i, 2].set_title('theta = constructor\n- alpha_driver * driver_rating')

        positions = model_1_ppc_sim.stan_variable('y_ppc').flatten() + 1
        n_bins = np.arange(22) - 0.5
        axes[s_i, 3].hist(positions, bins=n_bins, rwidth=0.85, density=True, color=colors[5], alpha=0.85, label="Simulated Positions")
        axes[s_i, 3].set_xticks(range(22))
        axes[s_i, 3].set_xlim([0.5, 20.5])
        axes[s_i, 3].set_yticks([])
        axes[s_i, 3].set_title('Position')

    for i in range(4):
        axes[2, i].set_xlabel(['alpha_driver', 'year_constructor', 'theta', 'Position'][i])

    for ax_row in axes:
        for ax in ax_row:
            ax.tick_params(axis='both', which='major')
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)

    fig.suptitle("Prior Predictive Checks for Model 1", fontsize=18)
    fig.tight_layout(pad=2.0)
    plt.show()

In [ ]:
sigmas = [0.5, 0.7, 1]

#### The driver with the best results.

In [ ]:
driver_rating = 2.5

draw_plots_ppc_model_1(sigmas, driver_rating)

#### The driver with the average results.

In [ ]:
driver_rating = 0

draw_plots_ppc_model_1(sigmas, driver_rating)

#### The driver with the weakest results.

In [ ]:
driver_rating = -2.5

draw_plots_ppc_model_1(sigmas, driver_rating)

### Model 2 PPC

In [ ]:
model_2_ppc = CmdStanModel(stan_file='stan/model_2_ppc.stan')

In [ ]:
def draw_plots_ppc_model_2(sigmas, driver_rating):
    fig, axes = plt.subplots(3, 5, figsize=(20, 10))
    colors = ["#130582", "#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"]

    for s_i in range(3):
        sigma = {'sigma': sigmas[s_i], 'driver_rating': driver_rating}
        model_2_ppc_sim = model_2_ppc.sample(data=sigma, iter_warmup=1, fixed_param=True, seed=25062025)

        axes[s_i, 0].hist(model_2_ppc_sim.stan_variable('engine').flatten(), bins=100, density=True, color=colors[0], alpha=0.8)
        axes[s_i, 0].set_yticks([])
        axes[s_i, 0].set_title(f'engine ~ Normal(0, {sigmas[s_i]})')

        axes[s_i, 1].hist(model_2_ppc_sim.stan_variable('alpha_driver').flatten(), bins=100, density=True, color=colors[1], alpha=0.8)
        axes[s_i, 1].set_yticks([])
        axes[s_i, 1].set_title(f'alpha_driver ~ Normal(1, {sigmas[s_i]})')

        axes[s_i, 2].hist(model_2_ppc_sim.stan_variable('year_constructor').flatten(), bins=100, density=True, color=colors[2], alpha=0.8)
        axes[s_i, 2].set_yticks([])
        axes[s_i, 2].set_title(f'year_constructor ~ Normal(0, {sigmas[s_i]})')

        axes[s_i, 3].hist(model_2_ppc_sim.stan_variable('theta').flatten(), bins=100, density=True, color=colors[4], alpha=0.8)
        axes[s_i, 3].set_yticks([])
        axes[s_i, 3].set_title('theta = engine + alpha_constructor_year \n - alpha_driver * driver_rating')

        positions = model_2_ppc_sim.stan_variable('y_ppc').flatten() + 1
        n_bins = np.arange(22) - 0.5
        axes[s_i, 4].hist(positions, bins=n_bins, rwidth=0.85, density=True, color=colors[5], alpha=0.85, label="Simulated Positions")
        axes[s_i, 4].set_xticks(range(22))
        axes[s_i, 4].set_xlim([0.5, 20.5])
        axes[s_i, 4].set_yticks([])
        axes[s_i, 4].set_title('Position')

    for i in range(5):
        axes[2, i].set_xlabel(['engine', 'alpha_driver', 'year_constructor', 'theta', 'Position'][i])

    for ax_row in axes:
        for ax in ax_row:
            ax.tick_params(axis='both', which='major')
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)

    fig.suptitle("Prior Predictive Checks for Model 2", fontsize=18)
    fig.tight_layout(pad=2.0)
    plt.show()

In [ ]:
sigmas = [0.5, 0.7, 1.0]

#### The driver with the best results.

In [ ]:
driver_rating = 2.5

draw_plots_ppc_model_2(sigmas, driver_rating)

#### An average driver from the middle of the field.

In [ ]:
driver_rating = 0

draw_plots_ppc_model_2(sigmas, driver_rating)

#### The driver with the worst results.

In [ ]:
driver_rating = -2.5

draw_plots_ppc_model_2(sigmas, driver_rating)

### Prior Selection and Justification

The choice of normal priors followed van Kesteren and Bergkamp (2023), who demonstrated their suitability in hierarchical models of performance. Normal distributions were selected due to their symmetry, interpretability, and compatibility with multilevel modeling assumptions. For the driver's skill component, the prior was centered at μ = 0 with a standard deviation σ = 0.7 based on prior predictive checks. In our model, the transformed skill component was later rescaled (mean = 1) to match the domain-specific interpretation involving an alpha multiplier drawn from a half-normal distribution.


## Posterior analysis (model 1) [0-4 pts]
- were there any issues with the sampling? if there were what kind of ideas for mitigation were used [1 pt]
- are the samples from posterior predictive distribution analyzed [1 pt]
- are the data consistent with posterior predictive samples and is it sufficiently commented (if they are not then is the justification provided)
have parameter marginal disrtibutions been analyzed (histograms of individual parametes plus summaries, are they diffuse or concentrated, what can we say about values) [1 pt]

There was no issue with sampling.

In [ ]:
def plot_post_3(model_fit, drivers_names):
    fig, axes = plt.subplots(1, len(drivers_names), figsize=(5 * len(drivers_names), 4), sharey=True)

    n_bins = np.arange(22) - 0.5

    for d_i, d_name in enumerate(drivers_names):
        ax = axes[d_i]
        driver_id = driver_id_map[d_name]
        results = df[df['DriverId'] == driver_id]
        results_idx = results.index

        ax.hist((results['Position']).tolist(),
                bins=n_bins,
                rwidth=0.9,
                histtype='step',
                edgecolor='black',
                density=True,
                label='Observed')

        ax.hist(model_fit.stan_variable('y_hat').T[results_idx].flatten() + 1,
                bins=n_bins,
                rwidth=0.9,
                color='cornflowerblue',
                edgecolor='royalblue',
                alpha=0.7,
                density=True,
                label='Simulated')

        ax.set_xticks(range(22))
        ax.set_xlim([0, 21])
        ax.set_yticks([])
        ax.set_title(d_name.upper() + '\nfinishing positions (2020-2024)', fontsize=11)
        ax.set_xlabel('Position')
        ax.legend(loc='upper right', fontsize=8)

    fig.tight_layout()
    plt.show()

In [ ]:
model_1 = CmdStanModel(stan_file='stan/model_1.stan')

In [ ]:
model_1_data = {'N': len(df),
                'C': len([*team_id_map.values()]),
                'D': len([*driver_id_map.values()]),
                'driver_rating': ratings,
                'constructor': teams,                
                'driver': drivers,
                'position': df['Position'] - 1} 

model_1_fit = model_1.sample(data=model_1_data, seed=25062025, iter_warmup=1000)

In [ ]:
drivers_names = ['ocon', 'norris', 'hamilton']
plot_post_3(model_1_fit, drivers_names)

### Parameter Marginal Distributions

#### Alpha Driver distribution

In [ ]:
drivers_names = ['ocon', 'norris', 'hamilton']
fig, axes = plt.subplots(1, len(drivers_names), figsize=(5 * len(drivers_names), 4), sharey=True)

for d_i, d_name in enumerate(drivers_names):
    driver_id = driver_id_map[d_name]
    print(f"Driver: {d_name}, ID: {driver_id}")
    az.plot_posterior(model_1_fit, var_names=["alpha_driver"], coords={"alpha_driver_dim_0": [driver_id - 1]}, ax=axes[d_i], color='cornflowerblue', hdi_prob=0.89)
    axes[d_i].set_title(d_name.upper() + '\nposterior alpha_driver', fontsize=11)

fig.suptitle("Posterior alpha_driver distributions for Model 1")
fig.tight_layout()
plt.show()

In [ ]:
# d_name = 'ocon'
# w = model_1_fit.stan_variable('alpha_driver').T[driver_id_map[d_name] - 1].flatten()
# x = plt.hist(model_1_fit.stan_variable('alpha_driver').T[driver_id_map[d_name] - 1].flatten(), bins=100, color='cornflowerblue', edgecolor='royalblue', density=True)

#### Alpha constructor distribution

In [ ]:
constructor_names = ['mercedes', 'red_bull', 'haas']
fig, axes = plt.subplots(1, len(constructor_names), figsize=(5 * len(constructor_names), 4), sharey=True)

for c_i, c_name in enumerate(constructor_names):
    constructor_id = team_id_map[c_name]
    print(f"Constructor: {c_name}, ID: {constructor_id}")
    az.plot_posterior(model_1_fit, var_names=["alpha_constructor"], coords={"alpha_constructor_dim_0": [constructor_id - 1]}, ax=axes[c_i], color='cornflowerblue', hdi_prob=0.89)
    axes[c_i].set_title(c_name.upper() + '\nposterior alpha_constructor', fontsize=11)

fig.suptitle("Posterior alpha_constructor distributions for Model 1")
fig.tight_layout()
plt.show()

### Driver Rating 2023

In [ ]:
ratings_2023 = df[df['Season'] == season_id_map[2023]][['DriverId', 'Rating_all']].drop_duplicates()

In [ ]:
driver = az.summary(model_1_fit, var_names=['alpha_driver'], kind='stats', hdi_prob=.50)

driversID_in_2023 = df[df.loc[:, 'Season']==season_id_map[2023]]['DriverId'].unique()

processed_driver = {
    key.split('_')[-1].capitalize(): value
    for key, value in driver_id_map.items()
}

driver['Driver_id'] = processed_driver.values()
driver['Driver_name'] = processed_driver.keys()
driver_2023 = driver[driver['Driver_id'].isin(driversID_in_2023)]
drivers_in_2023 = driver_2023['Driver_name'].tolist()
driver_2023 = driver_2023.merge(ratings_2023, left_on='Driver_id', right_on='DriverId', how='left')
driver_2023['Skill_weighted_alpha'] = driver_2023['mean'] * driver_2023['Rating_all']
driver_2023['Skill_weighted_2_5'] = driver_2023['hdi_25%'] * driver_2023['Rating_all']
driver_2023['Skill_weighted_97_5'] = driver_2023['hdi_75%'] * driver_2023['Rating_all']
driver_sorted = driver_2023.sort_values(by=['Skill_weighted_alpha'], ascending=True)
driver_sorted.reset_index(drop=True, inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

for i, row in driver_sorted.iterrows():
    y = i
    x_mean = row['Skill_weighted_alpha']
    x_min = row['Skill_weighted_2_5']
    x_max = row['Skill_weighted_97_5']

    ax.hlines(y, x_min, x_max, color='gray', linewidth=2)

    ax.plot(x_mean, y, 'o', color='teal')

ax.axvline(0, color='black', linestyle='--', linewidth=1)
ax.set_yticks(range(len(driver_sorted)))
ax.set_yticklabels(driver_sorted['Driver_name'])
ax.set_xlabel(r"Skill-weighted $\alpha\_driver$ ($\alpha \times rating$)")
ax.set_title("2023 F1 driver skills")
plt.tight_layout()
plt.show()

### Driver skill trajectories

In [ ]:
driver_rating = df[['DriverId', 'Rating_all', 'Season']]
driver_model1 = driver.merge(driver_rating, left_on='Driver_id', right_on='DriverId', how='left')

In [ ]:
driver_model1['Skill_weighted_alpha'] = driver_model1['mean'] * driver_model1['Rating_all']
driver_model1['Skill_weighted_2_5'] = driver_model1['hdi_25%'] * driver_model1['Rating_all']
driver_model1['Skill_weighted_97_5'] = driver_model1['hdi_75%'] * driver_model1['Rating_all']

In [ ]:
n_seasons = df['Season'].nunique()
driver_season_counts = driver_model1.groupby('DriverId')['Season'].nunique()
drivers_in_all_seasons = driver_season_counts[driver_season_counts == n_seasons].index
df_filtered = driver_model1[driver_model1['DriverId'].isin(drivers_in_all_seasons)]

In [ ]:
g = sns.FacetGrid(df_filtered, col='Driver_name', col_wrap=4, height=3, sharey=True)

# Linia i cieniowanie przedziału niepewności
g.map_dataframe(
    sns.lineplot, x='Season', y='Skill_weighted_alpha', marker='o'
)
g.map_dataframe(
    plt.fill_between, 'Season', 'Skill_weighted_2_5', 'Skill_weighted_97_5', alpha=0.2
)
g.set_axis_labels("Season", "Skill")
g.set_titles("{col_name}")
g.fig.suptitle("Skill evolution by driver (higher is better)", y=1.03)
plt.tight_layout()
plt.show()

### Constructor Rating

In [ ]:
constructor = az.summary(model_1_fit, var_names=['alpha_constructor'], kind='stats', hdi_prob=.50)

processed_teams = {
    key.split('_')[-1].capitalize(): value
    for key, value in team_id_map.items()
}
processed_teams["Red_Bull"] = processed_teams.pop('Bull')
processed_teams["Racing_Bulls"] = processed_teams.pop('Rb')
processed_teams["Aston_Martin"] = processed_teams.pop('Martin')

constructor['Constructor_id'] = processed_teams.values()
constructor['Constructor_name'] = processed_teams.keys()

constructor_list = constructor['Constructor_name'].tolist()

constructor_sorted = constructor.sort_values(by=['mean'], ascending=True)
constructor_sorted.reset_index(drop=True, inplace=True)

In [ ]:
constructor_color = {'Ferrari': 'red', 'Mercedes': 'turquoise', 'Red_Bull': 'blue', 
                     'Williams': 'deepskyblue', 'Aston_Martin': 'darkgreen', 
                     'Racing_Bulls': 'mediumblue', 'Alpine': 'royalblue', 
                     'Sauber': 'lightgreen', 'Mclaren': 'orange', 'Haas': 'k'}

fig, ax = plt.subplots(figsize=(8, 6))

for i, row in constructor_sorted.iterrows():
    y = i
    x_mean = row['mean']
    x_min = row['hdi_25%']
    x_max = row['hdi_75%']

    ax.hlines(y, x_min, x_max, color=constructor_color[row['Constructor_name']], linewidth=2)

    ax.plot(x_mean, y, 'o', color='teal')

ax.set_yticks(range(len(constructor_sorted)))
ax.set_yticklabels(constructor_sorted['Constructor_name'], fontsize=10)

ax.set_xlabel("Skill (lower is better)")
ax.set_title("F1 Constructors Skill Ranking", fontsize=14)

plt.tight_layout()
plt.show()

### Contribution of Constructor vs. Driver to Performance Variance

In [ ]:
fit_1 = az.from_cmdstanpy(posterior=model_1_fit,
                          log_likelihood='log_lik',
                          posterior_predictive='y_hat',
                          observed_data=model_1_data['position'])

In [ ]:
posterior = az.extract(fit_1, var_names=["alpha_constructor", "alpha_driver"], combined=True)

In [ ]:
var_constructors = np.var(posterior["alpha_constructor"], axis=1).mean()
var_drivers = np.var(posterior["alpha_driver"], axis=1).mean()

In [ ]:
total = var_constructors + var_drivers

constructor_share = var_constructors / total
driver_share = var_drivers / total

print(f"Constructor contribution to variance: {constructor_share:.2%}")
print(f"Driver contribution to variance:      {driver_share:.2%}")

The results indicate that:
- Constructor contribution to variance: 23.66%
- Driver contribution to variance: 76.34%

This means that, according to the Bayesian model, driver skill has a significantly greater impact on race results than the constructor (car/team) itself.

## Posterior analysis (model 2) [0-4 pts]
- were there any issues with the sampling? if there were what kind of ideas for mitigation were used [1 pt]
- are the samples from posterior predictive distribution analyzed [1 pt]
- are the data consistent with posterior predictive samples and is it sufficiently commented (if they are not then is the justification provided)
have parameter marginal disrtibutions been analyzed (histograms of individual parametes plus summaries, are they diffuse or concentrated, what can we say about values) [1 pt]

In [ ]:
model_2 = CmdStanModel(stan_file='stan/model_2.stan')

In [ ]:
model_2_data = {'N': len(df),
                'C': len([*team_id_map.values()]),
                'E': len([*engine_id_map.values()]),
                'D': len([*driver_id_map.values()]),
                'Y': len([*season_id_map.values()]),
                'driver_rating': ratings_by_year,
                'engine': engines,
                'constructor': teams,                
                'driver': drivers,
                'year': seasons,
                'position': df['Position'] - 1} 

model_2_fit = model_2.sample(data=model_2_data, seed=25062025, iter_warmup=1000)

In [ ]:
drivers_names = ['leclerc', 'norris', 'hamilton']
plot_post_3(model_2_fit, drivers_names)

### Parameter Marginal Distributions

Alpha Driver distribution

In [ ]:
drivers_names = ['leclerc', 'norris', 'hamilton']
fig, axes = plt.subplots(1, len(drivers_names), figsize=(5 * len(drivers_names), 4), sharey=True)

for d_i, d_name in enumerate(drivers_names):
    driver_id = driver_id_map[d_name]
    print(f"Driver: {d_name}, ID: {driver_id}")
    az.plot_posterior(model_2_fit, var_names=["alpha_driver"], coords={"alpha_driver_dim_0": [driver_id - 1]}, ax=axes[d_i], color='cornflowerblue', hdi_prob=0.89)
    axes[d_i].set_title(d_name.upper() + '\nposterior alpha_driver', fontsize=11)

fig.suptitle("Posterior alpha_driver distributions for Model 2")
fig.tight_layout()
plt.show()

Alpha Construtor year distribution

In [ ]:
x = model_2_fit.stan_variable('alpha_constructor_year')

In [ ]:
constructor_names = ['ferrari', 'mercedes', 'haas']
year = 2022
fig, axes = plt.subplots(1, len(constructor_names), figsize=(5 * len(constructor_names), 4), sharey=True)

for c_i, c_name in enumerate(constructor_names):
    constructor_id = team_id_map[c_name]
    print(f"Constructor: {c_name}, ID: {constructor_id}")
    az.plot_posterior(model_2_fit, var_names=["alpha_constructor_year"], coords={"alpha_constructor_year_dim_0": [season_id_map[year] - 1], "alpha_constructor_year_dim_1": [constructor_id - 1]}, ax=axes[c_i], color='cornflowerblue', hdi_prob=0.89)
    axes[c_i].set_title(c_name.upper() + '\nposterior alpha_constructor_year in ' + str(year), fontsize=11)

fig.suptitle("Posterior alpha_constructor_year distributions for Model 2")
fig.tight_layout()
plt.show()

Alpha Engine distribution

In [ ]:
engine_names = ['mercedes', 'ferrari', 'renault', 'rbpt']
fig, axes = plt.subplots(1, len(engine_names), figsize=(5 * len(engine_names), 4), sharey=True)

for c_i, c_name in enumerate(engine_names):
    engine_id = engine_id_map[c_name]
    print(f"Constructor: {c_name}, ID: {engine_id}")
    az.plot_posterior(model_2_fit, var_names=["alpha_engine"],coords={"alpha_engine_dim_0": [engine_id - 1]}, ax=axes[c_i], color='cornflowerblue', hdi_prob=0.89)
    axes[c_i].set_title(c_name.upper() + '\nposterior alpha_engine', fontsize=11)

fig.suptitle("Posterior alpha_engine distributions for Model 2")
fig.tight_layout()
plt.show()

### Driver skill trajectories

In [ ]:
driver = az.summary(model_2_fit, var_names=['alpha_driver'], kind='stats', hdi_prob=.50)
driver['Driver_id'] = driver_id_map.values()
driver['Driver_name'] = driver_id_map.keys()

In [ ]:
driver_rating = df[['DriverId', 'Rating_by_year', 'Season']]
driver_model2 = driver.merge(driver_rating, left_on='Driver_id', right_on='DriverId', how='left')

In [ ]:
driver_model2['Skill_weighted_alpha'] = driver_model2['mean'] * driver_model2['Rating_by_year']
driver_model2['Skill_weighted_2_5'] = driver_model2['hdi_25%'] * driver_model2['Rating_by_year']
driver_model2['Skill_weighted_97_5'] = driver_model2['hdi_75%'] * driver_model2['Rating_by_year']

In [ ]:
driver_season_counts = driver_model2.groupby('DriverId')['Season'].nunique()
drivers_in_all_seasons = driver_season_counts[driver_season_counts == n_seasons].index
df_filtered = driver_model1[driver_model2['DriverId'].isin(drivers_in_all_seasons)]

In [ ]:
g = sns.FacetGrid(df_filtered, col='Driver_name', col_wrap=4, height=3, sharey=True)

# Linia i cieniowanie przedziału niepewności
g.map_dataframe(
    sns.lineplot, x='Season', y='Skill_weighted_alpha', marker='o'
)
g.map_dataframe(
    plt.fill_between, 'Season', 'Skill_weighted_2_5', 'Skill_weighted_97_5', alpha=0.2
)
g.set_axis_labels("Season", "Skill")
g.set_titles("{col_name}")
g.fig.suptitle("Skill evolution by driver", y=1.03)
plt.tight_layout()
plt.show()

### Constructor Rating

In [ ]:
import re

def extract_indices(name):
    match = re.match(r"alpha_constructor_year\[(\d+), (\d+)\]", name)
    if match:
        return int(match.group(1)), int(match.group(2))
    else:
        return None, None

In [ ]:
constructor = az.summary(model_2_fit, var_names=['alpha_constructor_year'], kind='stats', hdi_prob=.50)
constructor = constructor.reset_index()

constructor[["year_idx", "constructor_idx"]] = constructor["index"].apply(
    lambda x: pd.Series(extract_indices(x))
)

team_index_to_name = {v - 1: k for k, v in processed_teams.items()}
year_index_to_name = {v - 1: k for k, v in season_id_map.items()}

constructor['Constructor_name'] = constructor["constructor_idx"].map(team_index_to_name)
constructor['Year'] = constructor["year_idx"].map(year_index_to_name)

constructor_list = constructor['Constructor_name'].tolist()

constructor_2023 = constructor[constructor['Year'] == 2023]

constructor_sorted = constructor_2023.sort_values(by=['mean'], ascending=True)
constructor_sorted.reset_index(drop=True, inplace=True)

In [ ]:
constructor_color = {'Ferrari': 'red', 'Mercedes': 'turquoise', 'Red_Bull': 'blue', 
                     'Williams': 'deepskyblue', 'Aston_Martin': 'darkgreen', 
                     'Racing_Bulls': 'mediumblue', 'Alpine': 'royalblue', 
                     'Sauber': 'lightgreen', 'Mclaren': 'orange', 'Haas': 'k'}

fig, ax = plt.subplots(figsize=(8, 6))

for i, row in constructor_sorted.iterrows():
    y = i
    x_mean = row['mean']
    x_min = row['hdi_25%']
    x_max = row['hdi_75%']

    ax.hlines(y, x_min, x_max, color=constructor_color[row['Constructor_name']], linewidth=2)

    ax.plot(x_mean, y, 'o', color='teal')

ax.set_yticks(range(len(constructor_sorted)))
ax.set_yticklabels(constructor_sorted['Constructor_name'], fontsize=10)

ax.set_xlabel("Skill (lower is better)")
ax.set_title("F1 Constructors Skill Ranking", fontsize=14)

plt.tight_layout()
plt.show()

### Constructor trajectories

In [ ]:
g = sns.FacetGrid(constructor, col='Constructor_name', col_wrap=4, height=3, sharey=True)

# Linia i cieniowanie przedziału niepewności
g.map_dataframe(
    sns.lineplot, x='Year', y='mean', marker='o'
)
g.map_dataframe(
    plt.fill_between, 'Year', 'hdi_25%', 'hdi_75%', alpha=0.2
)
g.set_axis_labels("Year", "Skill")
g.set_titles("{col_name}")
g.fig.suptitle("Skill evolution by constructor (lower is better)", y=1.03)
plt.tight_layout()
plt.show()

In [ ]:
### Contribution of Constructor vs. Driver to Performance Variance

In [ ]:
fit_2 = az.from_cmdstanpy(posterior=model_2_fit,
                          log_likelihood='log_lik',
                          posterior_predictive='y_hat',
                          observed_data=model_2_data['position'])

In [ ]:
posterior = az.extract(fit_2, var_names=["alpha_constructor_year", "alpha_driver", 'alpha_engine'], combined=True)

In [ ]:
var_constructors = np.var(posterior["alpha_constructor_year"], axis=1).mean()
var_drivers = np.var(posterior["alpha_driver"], axis=1).mean()
var_engines = np.var(posterior["alpha_engine"], axis=1).mean()

In [ ]:
total = var_constructors + var_drivers + var_engines

constructor_share = var_constructors / total
driver_share = var_drivers / total
engine_share = var_engines / total

print(f"Constructor contribution to variance: {constructor_share:.2%}")
print(f"Driver contribution to variance:      {driver_share:.2%}")
print(f"Engine contribution to variance:      {driver_share:.2%}")

In the second model, the variance contributions to race results are as follows:
- Constructor (team/car) contributes 78.78%,
- Driver contributes only 9.39%,
- Engine contributes 9.39%.

This means that, according to this model, the constructor plays the dominant role in determining race outcomes, while driver skill and engine have much smaller impacts.

Comparing the two models, we observe a significant difference in the importance of features. In the first model, the driver plays a much greater role, while in the second model, the team that builds the car dominates. From our expert point of view, we maintain that the car has a greater impact than the driver.

## Model comaprison [0-4 pts]
- Have models been compared using information criteria [1 pt]
- Have result for WAIC been discussed (is there a clear winner, or is there an overlap, were there any warnings) [1 pt]
- Have result for PSIS-LOO been discussed (is there a clear winner, or is there an overlap, were there any warnings) [1 pt]
- Whas the model comparison discussed? Do authors agree with information criteria? Why in your opinion one model better than another [1 pt]

In [ ]:
dict_compare = {
    'Model 1': fit_1,
    'Model 2': fit_2
}

#### Waic comparison

In [ ]:
waic_compare = az.compare(dict_compare, ic='waic')
print(waic_compare)

In [ ]:
az.plot_compare(waic_compare, figsize=(6, 4))
plt.title('WAIC comparision between Model 1 and Model 2 (higher is better)')
plt.show()

Based on the comparison of models using the WAIC criterion, we concluded that model 2 is better. First, it has a lower predictive loss—the difference is 292.8 in favor of model 2.

Next, we checked whether this difference is statistically significant. To do this, we calculated the ratio of the elpd_diff value to dse, where dse is the standard error of this difference. This ratio indicates how many times greater the observed difference is compared to its statistical uncertainty. In our case, it equals 4.36, which means that model 2 generalizes significantly better than model 1.

The p_waic value reflects the model’s complexity. We observe that model 2 is more complex but also generalizes better. This indicates that adding additional parameters improved its performance.

A warning appeared for both models, suggesting that the WAIC calculations may be unreliable. This is likely due to the large number of parameters in both cases.

#### Psis-loo comparison

In [ ]:
loo_compare = az.compare(dict_compare, ic='loo')
print(loo_compare)

In [ ]:
az.plot_compare(loo_compare, figsize=(6, 4))
plt.title('LOO comparision between Model 1 and Model 2 (higher is better)')
plt.show()

Based on the comparison of models using the PSIS-LOO criterion, we concluded that model 2 performs better. Model 2 has a higher expected log predictive density (elpd_loo = -8219.82) compared to model 1 (elpd_loo = -8512.51), resulting in an elpd difference of 292.7 in favor of model 2.

To evaluate the statistical significance of this difference, we calculated the ratio of elpd_diff to its standard error (dse). Here, the standard error of the difference is 67.19, which gives a ratio of approximately 4.36, indicating that the difference is significant and model 2 generalizes better than model 1.

The effective number of parameters (p_loo) is higher for model 2 (320.82 vs. 164.55), suggesting that model 2 is more complex but still achieves better predictive performance, indicating that the additional parameters improved the model.

However, a warning was issued for model 2, indicating potential reliability issues with the PSIS-LOO estimate. This warning often points to problematic observations or influential data points (high Pareto k values)

## Summary

For both information criteria, we obtained similar results that confirmed our expectations — the more complex model performs better, as we anticipated. Additionally, model 2, by incorporating information about the engine and how a given team performed in a specific year, allows for more accurate predictions regarding whether a change of engine in a particular team or a driver in a given year could improve the results.

## Predictions

In [ ]:
driver = 'stroll'
team = 'red_bull'
engine = 'rbpt'
season = 2024

results = model_2_fit.draws_pd()
driver_id  = driver_id_map[driver]
constructor_id = team_id_map[team]
engine_id = engine_id_map[engine]
year_id = season_id_map[season]
rating = df[(df['DriverId'] == driver_id) & (df['Season'] == year_id)]['Rating_by_year'].values[0]

model = (
    results[f'alpha_constructor_year[{year_id},{constructor_id}]'] 
    + results[f'alpha_engine[{engine_id}]'] 
    - results[f'alpha_driver[{driver_id}]'] * rating
)
model_list = model.tolist()
prob = expit(model_list) 
result = binomial(n=19, p=prob)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 5))

# pos_min = 1, pos_max = 20
n_bins = np.arange(22) - 0.5

ax.hist(result+1, bins=n_bins, rwidth=1, color='cornflowerblue', edgecolor='royalblue', density=True, alpha=1)
ax.set_xticks(range(22))
ax.set_xlim([0.5, 20.5])
ax.set_yticks([])
ax.set_title(f'{season} {driver} in {season} {team} with {engine} engine')
ax.set_xlabel(r'position')

fig.tight_layout()
plt.show()

## Bibliography

van Kesteren, E.-J., & Bergkamp, T. (2023). Bayesian analysis of Formula One race results: Disentangling driver skill and constructor advantage. Journal of Quantitative Analysis in Sports, 19(4), 273–293. https://doi.org/10.1515/jqas-2022-0021